In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import bigquery
client = bigquery.Client()


In [2]:
%%bigquery my_panda_data_frame

select start_station_name, end_station_name
from `bigquery-public-data.san_francisco.bikeshare_trips`
where start_station_name <> end_station_name
limit 10

In [ ]:
#Commuter trips
#Generate the table as a Pandas dataframe and summarize: number of trips, table of top stations